In [2]:
!pip install gradio 
!pip install torch 
!pip install transformers 
!pip install langdetect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 40.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 103.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.7 MB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=c7092778fe34e53132c9d894934c3c5267643e64a005798cee085eee617b1f62
  Stored in directory: /root/.cache/pip/wheels/

In [3]:
!pip install -q transformers accelerate bitsandbytes langdetect translate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.4 MB/s eta 0:00:00:00:0100:01


In [4]:
!pip install translate

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from langdetect import detect
from translate import Translator
import gradio as gr

In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

from huggingface_hub import login
login(hf_token)


In [7]:
model_name = "microsoft/phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
def detect_and_translate(query):
    try:
        lang = detect(query)
        if lang != 'en':
            translator = Translator(to_lang='en')
            return translator.translate(query), lang
        return query, 'en'
    except:
        return query, 'en'

In [21]:
def chat_with_ai(user_input):
    if not user_input:
        return "Please enter a medical question."

    # Detect language
    lang = detect(user_input)
    original_lang = lang  # Save for later translation back

    # Translate input to English if needed
    if lang != "en":
        translator = Translator(from_lang=lang, to_lang="en")
        user_input = translator.translate(user_input)

    # Strong system prompt to enforce medical response style
    prompt = (
        "You are a medical AI assistant. Provide direct, concise medical advice. "
        "DO NOT generate stories, dialogues, or casual conversations.\n\n"
        f"User Question: {user_input}\nMedical Answer:"
    )

    # Tokenize and generate response
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs, 
        max_new_tokens=50,  # Only generate new response tokens
        temperature=0.1,  # Slight randomness for natural answers
        top_p=0.9,  # Keeps output balanced
        do_sample=True  # Required for temperature and top_p to work
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove prompt text from response (if repeated)
    response = response.replace(prompt, "").strip()

    # Translate response back if needed
    if original_lang != "en":
        translator = Translator(from_lang="en", to_lang=original_lang)
        response = translator.translate(response)

    return response

In [24]:
demo = gr.Interface(
    fn=chat_with_ai,
    inputs=gr.Textbox(lines=2, placeholder="Enter your medical question..."),
    outputs=gr.Textbox(),
    title="Medical AI Chatbot",
    description="Ask a medical question and get an AI-generated response. This chat bot supports English, Urdu and Arabic Languages",
)

In [23]:
def launch_interface():
    try:
        demo.launch(server_port=7860, share=True)
    except OSError:
        print("Port 7860 busy, trying an alternative port...")
        demo.launch(share=True)  # Automatically picks an available port

# Run the app
if __name__ == "__main__":
    launch_interface()

Port 7860 busy, trying an alternative port...
* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://28867728c692815b65.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!nvidia-smi  # If using GPU
!free -h     # Check RAM usage
